In [1]:
!pip install translate

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
import spacy
from translate import Translator
import re

In [3]:
df = pd.read_excel('Survey_SleepHabits.xlsx')

In [4]:
df.columns

Index(['طابع زمني', '‎1. ‭What is your age ?', '‎2. ‭What is your gender ?',
       '‎3. ‭What is your occupation ?', '‎4. ‭What is your marital status ?',
       '‎5. ‭How many hours do you sleep on average per night ?',
       '‎6. ‭At what time do you usually go to bed ?',
       '‎7. ‭How would you rate the quality of your sleep ?',
       '‎8. ‭What do you feel on average when you wake up ?',
       '‎9. ‭How often do you feel sleepy during the daytime ?',
       '‎10. ‭Are there any factors that interrupt your sleep or make you unable sleep ? (e.g., sound, light, room temperature, etc...)',
       '‎11. ‭If so, explain what factor(s) and how it affects your sleep ?',
       '‎12. ‭Is your sleep often interrupted resulting in waking up in the middle of the night ?',
       '‎13. ‭If so, what do you do to go back to sleep ?',
       '‎14. ‭What activities do you usually do before bedtime ? (select all that apply)',
       '‎15. ‭What do you think about these activities' effect, is 

In [5]:
Data = df.iloc[:,[11,13,15,17,19,20]]

In [6]:
Data.columns

Index(['‎11. ‭If so, explain what factor(s) and how it affects your sleep ?',
       '‎13. ‭If so, what do you do to go back to sleep ?',
       '‎15. ‭What do you think about these activities' effect, is it positive or negative on your sleep ?',
       '‎17. ‭If you have difficulty falling asleep, what do you do to fall asleep ? (do you force yourself to sleep, listen to music/podcast or do other activities). Please explain',
       '‎19. ‭If you have any sleep disorders, what do you do to overcome it and to cope with it ?',
       '‎20. ‭If you use electronic devices (e.g., phone, TV, laptop, etc...)\n? how many hours do you spend using them'],
      dtype='object')

In [23]:
colum1 = Data.iloc[:,[0]]

In [8]:
## translate to english and fill Na

In [32]:
translator = Translator(from_lang='ar',to_lang="en")
for col in colum1.values:
    translation = translator.translate(str(col[0]))
    print(translation)
    

Street noise
Light disturb my sleep
nan
Dieting
light make me take more time to fall in sleep same for sound too
nan
Of course the sounds make me unable to sleep and this affects the duration of my sleep and the time I wake up, I feel a headache and start my day upset and also if the temperature is high I cannot sleep well
Time I sleep at ,noise,bad dreams
I affected if there is a sound or light.  I wake up to solve this problem and return to sleep.
Loud sound and light disrupt my sleep
Stress and anxiety and overthinking
nan
nan
nan
There's no factors
nan
Alarms
nan
I need to listen to musik or watch something to sleep
nan
Light and sound make sleep go away from me
nan
I can't sleep with the light on.
Minimum
nan
No
7 hours
الدرعية
nan
Sounds of any thing make me wake up and light ,open the door
nan
nan
nan
nan
Hot room temperature in summer
I wake up from the slightest noise , light pretty much any factor
No
.
Light and noise are the two biggest factors for not sleeping.
Make me tire

In [27]:
colum1.values

array([['Street noise '],
       ['Light disturb my sleep '],
       [nan],
       ['Light '],
       ['light make me take more time to fall in sleep same for sound too'],
       [nan],
       ['Of course the sounds make me unable to sleep and this affects the duration of my sleep and the time I wake up, I feel a headache and start my day upset and also if the temperature is high I cannot sleep well'],
       ['Time I sleep at ,noise,bad dreams '],
       ['I affected if there is a sound or light.  I wake up to solve this problem and return to sleep.  usually I do not remeber that '],
       ['Loud sound and light disrupt my sleep'],
       ['Stress and anxiety and overthinking'],
       [nan],
       [nan],
       [nan],
       ["There's no factors "],
       [nan],
       ['Alarms'],
       [nan],
       ['I need to listen to musik or watch somthing to sleep '],
       [nan],
       ['الضوء و الصوت تجعل النوم يذهب مني'],
       [nan],
       ['لا استطيع النوم ق الضوء'],
       ['soun